## 1. Setup and Dependencies

### 1.1 Install Libraries

In [2]:
# Install necessary libraries for MLflow and S3 artifact storage
%pip install -q mlflow boto3 awscli

Note: you may need to restart the kernel to use updated packages.


### 1.3 Import Libraries

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

## 2. MLflow Configuration

In [4]:
# Set the remote tracking server URI
mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

# Set or create a new experiment
mlflow.set_experiment("Experiment 2 - BoW vs TfIdf")

2025/11/12 12:01:32 INFO mlflow.tracking.fluent: Experiment with name 'Experiment 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlfow-bucket-2025/111723537539582833', creation_time=1762929091542, experiment_id='111723537539582833', last_update_time=1762929091542, lifecycle_stage='active', name='Experiment 2 - BoW vs TfIdf', tags={}>

## 3. Data Loading and Preparation

In [29]:
df = pd.read_csv('../data/reddit_preprocessing.csv')
# Display the shape after final cleaning
df.shape

(36662, 2)

In [30]:
print(df.isnull().sum())


clean_comment    0
category         0
dtype: int64


## 4. Experiment Function Definition

In [31]:
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    """Runs a single experiment, logs parameters and metrics to MLflow."""
    
    # 1. Vectorizer Initialization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    # 2. Data Split (Using original DataFrame to ensure consistent splitting)
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], 
                                                              test_size=0.2, random_state=42, stratify=df['category'])

    # 3. Vectorization
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # 4. MLflow Run
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # 5. Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report details
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # 6. Log confusion matrix plot
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close() # Close plot to free memory

        # 7. Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

        print(f"Completed run: {vectorizer_name} with {ngram_range} n-gram range. Accuracy: {accuracy:.4f}")

## 5. Execute Experiments

In [32]:
# Define the experiment space
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # Unigrams, Bigrams, Trigrams
max_features = 5000  # Limiting feature size for all experiments

for ngram_range in ngram_ranges:
    print(f"\n--- Running n-gram range: {ngram_range} ---")
    
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


--- Running n-gram range: (1, 1) ---


2025/11/12 13:05:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 13:06:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: BoW with (1, 1) n-gram range. Accuracy: 0.6460
🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833/runs/242417b4a2264e5f8204bc2f680f3b3c
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833


2025/11/12 13:07:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 13:07:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: TF-IDF with (1, 1) n-gram range. Accuracy: 0.6443
🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833/runs/deeb08102d6a48dcb74ba3e142ed1923
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833

--- Running n-gram range: (1, 2) ---


...

Completed run: TF-IDF with (1, 2) n-gram range. Accuracy: 0.6529
🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833/runs/95e50f6e8b69499bbed4f5aaca565d5f
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833

--- Running n-gram range: (1, 3) ---


2025/11/12 13:10:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 13:11:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: BoW with (1, 3) n-gram range. Accuracy: 0.6469
🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833/runs/558cf0a3c6aa4a7e82e305dc4c598f88
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833


2025/11/12 13:12:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/12 13:12:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Completed run: TF-IDF with (1, 3) n-gram range. Accuracy: 0.6471
🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833/runs/65353a8cc4cc4eff97012e3cacf58d76
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/111723537539582833


## 6. Conclusion
Review the results in the MLflow UI (using the tracking URI provided) to compare the performance metrics (accuracy, precision, recall, f1-score) logged for each combination of vectorizer type and n-gram range.